In [1]:
import pandas as pd
import spotipy
from secrets import secrets
from spotipy.oauth2 import SpotifyOAuth

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=secrets["Client Id"],
                                               client_secret=secrets["Client Secret"],
                                               redirect_uri="http://localhost:8892/callback",
                                               scope="user-top-read"))

In [3]:
ranges = {'short_term': 'Short term', 'medium_term': 'Medium term', 'long_term': 'Long term'}
limit = 20

df_top_artists = pd.DataFrame(columns=['Rank', 'Artist', 'Genres', 'Timeframe'])
df_top_tracks = pd.DataFrame(columns=['Rank', 'Track', 'Artist', 'Album', 'Duration', 'Timeframe'])

In [4]:
# Get top artists
for sp_range in ranges.keys():
    results = sp.current_user_top_artists(time_range=sp_range, limit=limit)
    for i, item in enumerate(results['items']):
        df_top_artists.loc[len(df_top_artists)] = [i+1, item['name'], item['genres'], ranges[sp_range]]

In [5]:
df_top_artists.loc[df_top_artists['Timeframe'] == 'Short term']

,Rank,Artist,Genres,Timeframe
0,1,Jay Chou,"[c-pop, mandopop, taiwan pop, zhongguo feng]",Short term
1,2,Kanye West,"[chicago rap, rap]",Short term
2,3,Kid Cudi,"[ohio hip hop, pop rap, rap]",Short term
3,4,Joji,"[alternative r&b, viral pop]",Short term
4,5,The Weeknd,"[canadian contemporary r&b, canadian pop, pop]",Short term
5,6,Frank Ocean,"[alternative r&b, hip hop, lgbtq+ hip hop, neo...",Short term
6,7,"Tyler, The Creator","[hip hop, rap]",Short term
7,8,Childish Gambino,"[atl hip hop, hip hop, pop rap, rap]",Short term
8,9,Chance the Rapper,"[chicago rap, conscious hip hop, hip hop, pop ...",Short term
9,10,The Chainsmokers,"[dance pop, edm, electropop, pop, pop dance, t...",Short term


In [6]:
df_top_artists.loc[df_top_artists['Timeframe'] == 'Medium term']

,Rank,Artist,Genres,Timeframe
20,1,Jay Chou,"[c-pop, mandopop, taiwan pop, zhongguo feng]",Medium term
21,2,blink-182,"[pop punk, punk, socal pop punk]",Medium term
22,3,Kanye West,"[chicago rap, rap]",Medium term
23,4,San Holo,"[edm, electro house, electronic trap, electrop...",Medium term
24,5,Joji,"[alternative r&b, viral pop]",Medium term
25,6,Gorillaz,[alternative hip hop],Medium term
26,7,Frank Ocean,"[alternative r&b, hip hop, lgbtq+ hip hop, neo...",Medium term
27,8,Sum 41,"[alternative metal, canadian pop punk, canadia...",Medium term
28,9,Coldplay,"[permanent wave, pop]",Medium term
29,10,My Chemical Romance,"[emo, pop punk]",Medium term


In [7]:
df_top_artists.loc[df_top_artists['Timeframe'] == 'Long term']

,Rank,Artist,Genres,Timeframe
40,1,Kanye West,"[chicago rap, rap]",Long term
41,2,Jay Chou,"[c-pop, mandopop, taiwan pop, zhongguo feng]",Long term
42,3,Childish Gambino,"[atl hip hop, hip hop, pop rap, rap]",Long term
43,4,The Weeknd,"[canadian contemporary r&b, canadian pop, pop]",Long term
44,5,Young Thug,"[atl hip hop, atl trap, gangster rap, hip hop,...",Long term
45,6,The Chainsmokers,"[dance pop, edm, electropop, pop, pop dance, t...",Long term
46,7,Drake,"[canadian hip hop, canadian pop, hip hop, pop ...",Long term
47,8,A$AP Rocky,"[east coast hip hop, hip hop, rap, trap, under...",Long term
48,9,ILLENIUM,"[edm, electropop, melodic dubstep, pop, pop da...",Long term
49,10,BROCKHAMPTON,"[boy band, hip hop, rap, underground hip hop]",Long term


In [8]:
# Get top track
for sp_range in ranges:
    results = sp.current_user_top_tracks(time_range=sp_range, limit=limit)
    for i, item in enumerate(results['items']):
        track = item['name']
        artist = item['artists'][0]['name']
        album = item['album']['name']
        duration = int(item['duration_ms'])
        minute = int((duration / (1000*60)) % 60)
        second = int((duration / 1000) % 60)
        duration = str(minute) + ':' + str(second)
        df_top_tracks.loc[len(df_top_tracks)] = [i+1, track, artist, album, duration, ranges[sp_range]]

In [9]:
df_top_tracks.loc[df_top_tracks['Timeframe'] == 'Short term']

,Rank,Track,Artist,Album,Duration,Timeframe
0,1,安靜,Jay Chou,范特西,5:34,Short term
1,2,Collard Greens,ScHoolboy Q,Oxymoron (Deluxe),4:59,Short term
2,3,With You (Jai Wolf Remix),Dirty South,With You (The Remixes),5:28,Short term
3,4,All Day,Kanye West,All Day,5:10,Short term
4,5,Nikes,Frank Ocean,Blonde,5:14,Short term
5,6,算什麼男人,Jay Chou,哎呦 不錯哦,4:49,Short term
6,7,千里之外,Jay Chou,依然范特西,4:14,Short term
7,8,明明就,Jay Chou,十二新作,4:20,Short term
8,9,七里香,Jay Chou,七里香,4:55,Short term
9,10,青花瓷,Jay Chou,我很忙,3:57,Short term


In [10]:
df_top_tracks.loc[df_top_tracks['Timeframe'] == 'Medium term']

,Rank,Track,Artist,Album,Duration,Timeframe
20,1,安靜,Jay Chou,范特西,5:34,Medium term
21,2,With Me,Sum 41,Underclass Hero,4:51,Medium term
22,3,珊瑚海,Jay Chou,11月的蕭邦,4:14,Medium term
23,4,Welcome to the Black Parade,My Chemical Romance,The Black Parade,5:11,Medium term
24,5,Wake Me up When September Ends,Green Day,American Idiot,4:45,Medium term
25,6,明明就,Jay Chou,十二新作,4:20,Medium term
26,7,Fix You,Coldplay,X&Y,4:55,Medium term
27,8,算什麼男人,Jay Chou,哎呦 不錯哦,4:49,Medium term
28,9,Always,blink-182,Greatest Hits,4:17,Medium term
29,10,How to Save a Life,The Fray,How To Save A Life,4:22,Medium term


In [11]:
df_top_tracks.loc[df_top_tracks['Timeframe'] == 'Long term']

,Rank,Track,Artist,Album,Duration,Timeframe
40,1,Crew (feat. Brent Faiyaz & Shy Glizzy),GoldLink,At What Cost,2:56,Long term
41,2,"A$AP Forever REMIX (feat. Moby, T.I. & Kid Cudi)",A$AP Rocky,TESTING,5:15,Long term
42,3,晴天,Jay Chou,葉惠美,4:29,Long term
43,4,pick up the phone,Young Thug,Birds In The Trap Sing McKnight,4:12,Long term
44,5,Fight Night,Migos,No Label II,3:36,Long term
45,6,RNP (feat. Anderson .Paak),Cordae,The Lost Boy,2:55,Long term
46,7,Slide (feat. Frank Ocean & Migos),Calvin Harris,Slide (feat. Frank Ocean & Migos),3:50,Long term
47,8,OKRA,"Tyler, The Creator",OKRA,2:31,Long term
48,9,Icha Gicha (feat. Pharrell Williams),Kap G,SupaJefe,3:30,Long term
49,10,珊瑚海,Jay Chou,11月的蕭邦,4:14,Long term


In [12]:
sp.me()

{'display_name': 'Bin Xuan Kong',
 'external_urls': {'spotify': 'https://open.spotify.com/user/12120382831'},
 'followers': {'href': None, 'total': 53},
 'href': 'https://api.spotify.com/v1/users/12120382831',
 'id': '12120382831',
 'images': [{'height': None,
   'url': 'https://scontent-hkt1-2.xx.fbcdn.net/v/t1.0-1/p320x320/11988649_10205375733654944_669349554023656758_n.jpg?_nc_cat=110&ccb=2&_nc_sid=0c64ff&_nc_ohc=qNH7qntD3ukAX-nqGgf&_nc_ht=scontent-hkt1-2.xx&tp=6&oh=412236df77fa5d56b351efeffe5f7f89&oe=6009A752',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:12120382831'}